### Trimming Responses of Excess Output

In [ ]:
import re
def extract_main_class(code: str, few_shot: bool) -> str:
    if few_shot:
        n = 2

    elif not few_shot:
        n = 1

    starts = [m.start() for m in re.finditer(r'public class Main', code)]

    if len(starts) < n:
        return ""  

    start_index = starts[n - 1]  # 0-based indexing
    brace_count = 0
    in_class = False
    end_index = None

    for i in range(start_index, len(code)):
        char = code[i]

        if char == '{':
            brace_count += 1
            if not in_class:
                in_class = True
        elif char == '}':
            brace_count -= 1
            if in_class and brace_count == 0:
                end_index = i
                break

    if end_index is not None:
        return code[start_index:end_index + 1]
    else:
        return code[start_index:]  # fallback if not closed properly


import pandas as pd
df = pd.read_csv("/media/mujtaba/DATA/nick/UnitTestExamples/data/results/starcoder2-3b__nick/responses.csv")

# extract_main_class(df.iloc[0].iloc[0], few_shot=True)

trimmed = []
for row in df:
    untrimmed_response = row
    trimmed_response = extract_main_class(untrimmed_response, few_shot=True)
    trimmed.append(trimmed_response)

import os
os.mkdir("/media/mujtaba/DATA/nick/UnitTestExamples/data/clean_responses")
df.to_csv("/media/mujtaba/DATA/nick/UnitTestExamples/data/clean_responses/few_shot_java.csv")



In [25]:
import re

def extract_main_class(code: str, few_shot: bool) -> str:
    # Determine which instance to extract (2nd if few-shot, 1st otherwise)
    n = 2 if few_shot else 1

    # Find all indices where "public class Main" appears
    starts = [m.start() for m in re.finditer(r'public class Main', code)]
    
    if len(starts) < n:
        return ""  # Not enough occurrences

    start_index = starts[n - 1]  # Get the nth occurrence (1-based index)

    # Brace counting to find the matching closing }
    brace_count = 0
    in_class = False
    end_index = None

    for i in range(start_index, len(code)):
        char = code[i]
        if char == '{':
            brace_count += 1
            if not in_class:
                in_class = True
        elif char == '}':
            brace_count -= 1
            if in_class and brace_count == 0:
                end_index = i
                break

    # Return only the full class block, starting from the nth "public class Main"
    if end_index is not None:
        return code[start_index:end_index + 1].strip()
    else:
        return code[start_index:].strip()  # Fallback if closing brace is missing

In [ ]:
import pandas as pd
df = pd.read_csv("/media/mujtaba/DATA/nick/UnitTestExamples/data/results/starcoder2-3b__nick/responses.csv")

# extract_main_class(df.iloc[0].iloc[0], few_shot=True)

trimmed = []
for row in df:
    untrimmed_response = row.iloc[0]
    trimmed_response = extract_main_class(untrimmed_response, few_shot=False)
    trimmed.append(trimmed_response)

import os
os.mkdir("/media/mujtaba/DATA/nick/UnitTestExamples/data/clean_responses")
df.to_csv("/media/mujtaba/DATA/nick/UnitTestExamples/data/clean_responses/no_shot_java.csv")